#Imports

In [ ]:
#!pip install --quiet "pytorch-lightning>=1.3" "torchmetrics>=0.3" "torch>=1.6, <1.9" "torchvision"
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install pytorch-lightning
!pip install pickle5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl
import torchvision
import torchvision.models as visionmodels
import torchvision.transforms as transforms
import torch.utils.data.dataloader as DataLoader
import torch.utils.data.dataset as Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable,grad
from torchvision import datasets
from collections import *
import os
from torchvision.io import read_image
from datetime import datetime
import glob
from zipfile import ZipFile
from PIL import Image
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)
pwd = '/content/'
base_dir = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer'
os.chdir(base_dir) 
import models 
os.chdir(pwd)

#Globals

In [ ]:
 torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

FLAGS = {} 
FLAGS['datadir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/landscapes'
FLAGS['labeldir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/landscapes.csv'
FLAGS['batch_size'] = 8
FLAGS['test_batch_size'] = 2
FLAGS['learning_rate'] = .1
FLAGS['num_epochs'] = 2000
FLAGS['im_channels'] = 3
FLAGS['noise_scale_factor'] = .01
FLAGS['style_weight'] = 2e5
FLAGS['pixel_weight'] = 10. 
FLAGS['content_weight'] = 1.
FLAGS['tv_weight'] = 1. 
FLAGS['im_size'] = 256
FLAGS['val_image_path'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/big_sur.jpg' 
FLAGS['style_image_path'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/Paintings/swirl.png' 
FLAGS['val_size_mult'] = 2
FLAGS['output_path'] = 'outputs/Texture_net_BN'
FLAGS['model_path'] = 'models/Texture_net_BN'
FLAGS['output_fname'] = 'test_output_'
FLAGS['model_fname'] = 'landscape_model'
FLAGS['home_dir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer'

#Shortcuts

In [ ]:
'''
  LITDataModule(train_batch_size = , 
                val_batch_size = ,
                train_im_size = , 
                val_im_size = , 
                data_zip = [], 
                data_path = [], 
                train_anno_path = , 
                test_anno_path = , 
                val_anno_path = 
                )
  LITAdaINDatamodule(train_batch_size = , 
                val_batch_size = ,
                train_im_size = , 
                val_im_size = , 
                data_zip = [], 
                data_path = [], 
                train_anno_path = , 
                test_anno_path = , 
                val_anno_path = 
                )
  LITTextureNet(style_image : torch.Tensor,
                      batch_size : int = 8,
                      normalization_method: str = 'BatchNorm2d', 
                      num_layers : int = 6, 
                      base_feat_maps: int = 8, 
                      im_channels: int = 3,
                      vgg_net : str = 'vgg19', 
                      optimizer : str = 'Adam', 
                      learning_rate : float = .1, 
                      betas : tuple = (.5,.99),
                      noise_scale_factor = .01,
                      style_weight = 1e5, 
                      content_weight = 1.)
  LITFastNST(style_image : torch.Tensor,
                      batch_size : int = 8,
                      normalization_method: str = 'BatchNorm2d',
                      vgg_net : str = 'vgg19', 
                      optimizer : str = 'Adam', 
                      learning_rate : float = .1, 
                      betas : tuple = (.5,.99),
                      noise_scale_factor = .01,
                      style_weight = 1., 
                      content_weight = 1., 
                      pixel_weight = 10., 
                      tv_weight = 1.)
  LITAdaIN(batch_size : int = 8,
                      vgg_net : str = 'vgg19', 
                      optimizer : str = 'Adam', 
                      learning_rate : float = .1, 
                      betas : tuple = (.5,.99),
                      noise_scale_factor = .01,
                      style_weight = 1., 
                      content_weight = 1. )
    
  extract_features(data_dir,
                  output_fname,
                  vgg_net = 
                  im_size =   
                  )
  
'''

#Test Area

##Unzip Data

In [ ]:
# models.unzip_data(['data/cats','data/monet'],['/content/gdrive/MyDrive/Colab Notebooks/data/cats.zip', '/content/gdrive/MyDrive/Colab Notebooks/data/monet_jpg.zip'])


##Import data module

In [ ]:
dm = models.LITDataModule(train_batch_size = FLAGS['batch_size'] , 
                val_batch_size = FLAGS['test_batch_size'],
                train_im_size = FLAGS['im_size'], 
                val_im_size = FLAGS['im_size']*FLAGS['val_size_mult'], 
                data_zip = [], 
                data_path = [FLAGS['datadir'],FLAGS['datadir']], 
                train_anno_path = FLAGS['labeldir'], 
                test_anno_path = FLAGS['labeldir'], 
                val_anno_path = FLAGS['labeldir']
                )
# dm = models.LITAdaINDataModule(train_batch_size = FLAGS['batch_size'] , 
#                 val_batch_size = FLAGS['test_batch_size'],
#                 train_im_size = FLAGS['im_size'], 
#                 val_im_size = FLAGS['im_size']*FLAGS['val_size_mult'], 
#                 data_zip = [], 
#                 data_path = ['data/cats','data/monet'], 
#                 train_anno_path = '/content/gdrive/MyDrive/Colab Notebooks/data/monet.pkl', 
#                 test_anno_path = '/content/gdrive/MyDrive/Colab Notebooks/data/monet.pkl', 
#                 val_anno_path ='/content/gdrive/MyDrive/Colab Notebooks/data/monet.pkl'
#                 )

##Define Model

In [ ]:
transform = transforms.Compose([ transforms.Resize(FLAGS['im_size']), transforms.CenterCrop((FLAGS['im_size'],FLAGS['im_size']))])
val_transform = transforms.Compose([ transforms.Resize(FLAGS['im_size']*FLAGS['val_size_mult']), transforms.CenterCrop((FLAGS['im_size']*FLAGS['val_size_mult'],FLAGS['im_size']*FLAGS['val_size_mult']))])
style_image = transforms.ToTensor()(transform(Image.open(FLAGS['style_image_path']).convert('RGB'))).unsqueeze(0)
val_image = transforms.ToTensor()(transform(Image.open(FLAGS['val_image_path']).convert('RGB'))).unsqueeze(0)
texture_net = models.LITTextureNet(style_image,
                             val_image,
                      batch_size = FLAGS['batch_size'],
                      normalization_method  = 'BatchNorm2d', 
                      style_weight = 2e4, 
                      content_weight = 1e6)
johnson_nst = models.LITFastNST(style_image,
                          val_image,
                      batch_size = FLAGS['batch_size'],
                      normalization_method =  'BatchNorm2d',
                      vgg_net = 'vgg19', 
                      optimizer  = 'Adam', 
                      learning_rate  = .001, 
                      style_weight = .01, 
                      content_weight = 100., 
                      pixel_weight = 1000., 
                      tv_weight = .0001)
# model = models.LITAdaIN(batch_size  = FLAGS['batch_size'],
#                       vgg_net  = 'vgg16', 
#                       optimizer  = 'Adam', 
#                       learning_rate  = FLAGS['learning_rate'], 
#                       style_weight = 1., 
#                       content_weight = 1. )


##Define Logger and Trainer

In [ ]:
tb_logger_texture_net = pl_loggers.TensorBoardLogger("logs/texture_net")
tb_logger_nst = pl_loggers.TensorBoardLogger('logs/Fast_NST')
texture_net_trainer = pl.Trainer(gpus = 1, logger = tb_logger_texture_net, log_every_n_steps= 10, stochastic_weight_avg = True, 
                      callbacks = [EarlyStopping(monitor = 'style_loss',min_delta= 1e2, patience=25, verbose=True, mode="min", check_finite=True, stopping_threshold=1e5)])
nst_trainer = pl.Trainer(gpus = 1, logger = tb_logger_nst, log_every_n_steps= 10, stochastic_weight_avg = True, 
                      callbacks = [EarlyStopping(monitor = 'style loss',min_delta= .1, patience=25, verbose=True, mode="min", check_finite=True, stopping_threshold=1.0)])


##Run  Model

In [ ]:
#texture_net_trainer.fit(texture_net,dm)
nst_trainer.fit(johnson_nst,dm) 

In [ ]:
%load_ext tensorboard
%cd /content/
%tensorboard --logdir logs/texture_net

In [ ]:
%tensorboard --logdir logs/Fast_NST